## Split.io + OpenTelemetry + Lightstep

Feature flags, OpenTelemetry-style. Instruments Split.io Python SDK to connect feature flags to observability.

This example is a Python notebook, click each code cell in order to run.

### Setup

Install feature flags SDKs and OpenTelemetry dependencies. Set your API keys and tokens.



#### Install dependencies

In [ ]:

! pip install -q opentelemetry-api==0.17b0
! pip install -q opentelemetry-instrumentation==0.17b0
! pip install -q wrapt==1.12.1

! pip install -q splitio_client[cpphash]==8.4.0

# For configuring application/service code for OpenTelemetry with Lightstep
! pip uninstall -q -y protobuf
! pip install -q protobuf>=3.13.0
! pip install -q opentelemetry_launcher==0.17b0

#### Set Access Tokens

In [ ]:
# Signup here to get an access token: https://app.lightstep.com/signup/developer
# To find your token: https://docs.lightstep.com/docs/create-and-manage-access-tokens
import getpass
lightstep_access_token=getpass.getpass('Please paste your Lightstep access token: ')
lightstep_project = input('Please paste your Lightstep project name: ')

splitio_key = getpass.getpass('Please paste your Split.io access token: ')

#### Instrument Split.io Python SDK

*Use OpenTracing APIs to capture every call to `get_treatment` on the Split Python SDK -- no changes to the Split Python SDK are needed.*

Traditionally, this library-specific instrumentation code is distributed as an open-source package as part of [OpenTelemetry's contrib collection](https://github.com/open-telemetry/opentelemetry-python-contrib).

In [ ]:
import splitio

from opentelemetry import trace
from opentelemetry.instrumentation.instrumentor import BaseInstrumentor
from wrapt import ObjectProxy, wrap_function_wrapper
from pprint import pprint

_DEFAULT_SERVICE = "splitio"
version = "0.0.1"

def _traced_get_treatment(func, instance, args, kwargs):
  tracer = getattr(splitio, "_opentelemetry_tracer")
  with tracer.start_as_current_span(
        'get_treatment', kind=trace.SpanKind.CLIENT
    ) as span:
        return_val = func(*args, **kwargs)

        if span.is_recording():
            # Set attributes on a trace that describe the treatment
            span.set_attribute("split.io.key", args[0])
            span.set_attribute("split.io.name", args[1])
            span.set_attribute("split.io.value", return_val)
        return return_val

class SplitInstrumentor(BaseInstrumentor):
    """An instrumentor for Split.io
    See `BaseInstrumentor`
    """

    def _instrument(self, **kwargs):
        tracer_provider = kwargs.get(
            "tracer_provider", trace.get_tracer_provider()
        )
        setattr(
            splitio,
            "_opentelemetry_tracer",
            tracer_provider.get_tracer(_DEFAULT_SERVICE, version),
        )

        if splitio.__version__ == '8.4.0':
            wrap_function_wrapper(
                "splitio", "client.client.Client.get_treatment", _traced_get_treatment
            )


    def _uninstrument(self, **kwargs):
        if splitio.__version__ == '8.4.0':
            unwrap(splitio.client, "get_treatment")

### Configure OpenTelemetry for Lightstep

Use Lightstep's OpenTelemetry Python launcher to send traces to Lightstep.

In [ ]:
from opentelemetry.launcher import configure_opentelemetry

service_name = "tutorial-featureflags"

# This automatically configures OpenTelemetry to send data to Lightstep.
# Many other destinations are supported, see: https://pypi.org/search/?q=opentelemetry-exporter
configure_opentelemetry(
    service_name=service_name,
    access_token=lightstep_access_token,
)

### Run code with feature flags

This generates some telemetry by calling `get_treatment`. This simulates a server-side python application using Split treatments.

In [ ]:
import sys
import logging
import time
from random import randint, uniform

from opentelemetry import trace
tracer = trace.get_tracer('DonutsApp', '0.0.1')

from splitio import get_factory
from splitio.exceptions import TimeoutException

logging.basicConfig(level=logging.DEBUG)

config = {'ready' : 5000}

try:
  factory = get_factory(splitio_key, config=config)
  split = factory.client()
except TimeoutException:
  # The SDK failed to initialize in a second. Abort!
  sys.exit()

# Instrument split.io -- this part is important to get telemetry!
SplitInstrumentor().instrument()

# Mock server-side handler with random latency
# that responds to a customer-facing request
# Typically, this would be a flask app, etc.
def get_donuts_handler(customer_id):
  with tracer.start_as_current_span("GET /donuts"):
    time.sleep(uniform(0.2, 0.4))
    with tracer.start_as_current_span("get_donuts_handler"):
      # treatment is set to 50% on and 50% off
      treatment = split.get_treatment(customer_id, 'TEST_SPLIT')
      # fake database calls
      with tracer.start_as_current_span("database call"):
        if treatment == 'on':
          # slow
          time.sleep(uniform(2.5, 3.2))
        else:
          # fast
          time.sleep(uniform(0.5, 0.9))

print("Generating some requests, this should take about 5 minutes...")

# Simulate 10 server-side requests from random customers
for _ in range(100):
  get_donuts_handler(randint(0, 1000))
  

### View data in Lightstep

Connect feature flags to the latency and errors.

![Split.io Flags in Lightstep](./screenshots/splitio_lightstep.png)

In [ ]:
from IPython.core.display import display, HTML

display(HTML(f'<h3><a target="_blank" href="https://app.lightstep.com/{lightstep_project}/explorer?query=%22instrumentation.name%22%20IN%20%28%22splitio%22%29">Explore traces for Split.io in Lightstep</a></h3>'))